First, let's define the classification function. This function will take a user question as input, construct a prompt for the Gemini model to classify it, and then return the classified category.

In [1]:
!pip install pytest --quiet

Now, let's write unit tests for both `classify_question_gemini` and `generate_social_media_post` functions using `pytest`. Since these functions interact with the Gemini API, we'll use `unittest.mock` to simulate the Gemini model's responses. This allows us to test the logic of our functions without actually making API calls, making the tests faster and more reliable.

In [2]:
#First Block - The Start of it all

import pytest
from unittest.mock import MagicMock

# Mock the gemini_model globally for testing purposes
gemini_model = MagicMock()

def test_classify_question_gemini_employment():
    gemini_model.generate_content.return_value.text.strip.return_value = "Employment"
    assert classify_question_gemini("What are the benefits of working here?") == "Employment"

def test_classify_question_gemini_tax_related():
    gemini_model.generate_content.return_value.text.strip.return_value = "Tax Related"
    assert classify_question_gemini("How do I file my annual income tax return?") == "Tax Related"

def test_classify_question_gemini_general_information():
    gemini_model.generate_content.return_value.text.strip.return_value = "General Information"
    assert classify_question_gemini("What's the capital of France?") == "General Information"

def test_classify_question_gemini_emergency_services():
    gemini_model.generate_content.return_value.text.strip.return_value = "Emergency Services"
    assert classify_question_gemini("I need an ambulance right away!") == "Emergency Services"

def test_classify_question_gemini_unknown_category():
    gemini_model.generate_content.return_value.text.strip.return_value = "Random Category"
    assert classify_question_gemini("What is the meaning of life?") == "Unknown"

def test_classify_question_gemini_api_error():
    gemini_model.generate_content.side_effect = Exception("API error")
    assert classify_question_gemini("Any question") == "Unknown"
    gemini_model.generate_content.side_effect = None # Reset side effect

def test_generate_social_media_post_weather_emergency():
    mock_post = "#WeatherAlert Severe Thunderstorm Warning in Downtown Cityville until 8 PM EST. Seek shelter! #SafetyFirst"
    gemini_model.generate_content.return_value.text.strip.return_value = mock_post
    details = {
        "event": "Severe Thunderstorm Warning",
        "area": "Downtown Cityville",
        "time_period": "Until 8 PM EST",
        "instructions": "Seek shelter immediately. Avoid unnecessary travel."
    }
    assert generate_social_media_post("weather emergency", details) == mock_post

def test_generate_social_media_post_holiday_announcement():
    mock_post = "Happy National Day of Service on October 26th! Join us in volunteering and making a difference! #CommunityService"
    gemini_model.generate_content.return_value.text.strip.return_value = mock_post
    details = {
        "holiday_name": "National Day of Service",
        "date": "October 26th",
        "message": "Join us in volunteering and making a difference in our community!"
    }
    assert generate_social_media_post("holiday", details) == mock_post

def test_generate_social_media_post_school_closing():
    mock_post = "All City Schools closed tomorrow, January 15th, due to icy road conditions. Stay safe! #SchoolClosing #InclementWeather"
    gemini_model.generate_content.return_value.text.strip.return_value = mock_post
    details = {
        "school_name": "All City Schools",
        "reason": "Inclement weather and icy road conditions",
        "date": "Tomorrow, January 15th",
        "instructions": "Stay safe and warm! Online learning resources available."
    }
    assert generate_social_media_post("school closing", details) == mock_post

def test_generate_social_media_post_api_error():
    gemini_model.generate_content.side_effect = Exception("API error")
    details = {"event": "Test", "message": "Test"}
    assert generate_social_media_post("generic", details) == ""
    gemini_model.generate_content.side_effect = None # Reset side effect


# To run the tests, you would typically save this code as a Python file (e.g., test_functions.py)
# and then run 'pytest' from your terminal in the same directory.
# In a Colab notebook, you can run pytest directly using !pytest

# Run pytest within the notebook
print("\nRunning pytest...")
!pytest -s -v -k "test_classify_question_gemini or test_generate_social_media_post"


Running pytest...
============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-4.12.1, langsmith-0.6.6, typeguard-4.4.4
collected 0 items                                                              

============================ no tests ran in 0.01s =============================


In [3]:
# Challenge three: Testing and evaluation

def classify_question_gemini(question):
    """
    Classifies a user question into one of predefined categories using the Gemini API.

    Args:
        question (str): The user's question to classify.

    Returns:
        str: The classified category (Employment, General Information, Emergency Services, or Tax Related),
             or 'Unknown' if classification fails or is ambiguous.
    """
    categories = ["Employment", "General Information", "Emergency Services", "Tax Related"]
    prompt = f"""Classify the following question into one of these categories: {', '.join(categories)}.
    Return only the category name.
    Question: "{question}"
    Category:"""

    try:
        response = gemini_model.generate_content(prompt)
        # Gemini's response might contain extra text or newlines, so we'll strip it.
        classified_category = response.text.strip()

        # Optional: Add a check to ensure the classified category is one of the valid ones
        if classified_category in categories:
            return classified_category
        else:
            # If Gemini returns something not in our list, it's 'Unknown'
            print(f"Warning: Gemini returned an unexpected category: {classified_category}")
            return "Unknown"
    except Exception as e:
        print(f"Error classifying question: {e}")
        return "Unknown"

# Example usage of the function
print(classify_question_gemini("What are the benefits of working here?"))
print(classify_question_gemini("How do I file my annual income tax return?"))
print(classify_question_gemini("What's the capital of France?"))
print(classify_question_gemini("I need an ambulance right away!"))

Unknown
Unknown
Unknown
Unknown


Next, let's define a function that generates social media posts for various government announcements using the Gemini model. This function will take the announcement type and relevant details as input, and construct a prompt for Gemini to generate a suitable post.

In [4]:
# FUNCTION Conjunction

def generate_social_media_post(announcement_type, details):
    """
    Generates a social media post for a government announcement using the Gemini API.

    Args:
        announcement_type (str): The type of announcement (e.g., 'weather emergency', 'holiday', 'school closing').
        details (dict): A dictionary containing specific details for the announcement.

    Returns:
        str: The generated social media post.
    """

    # Construct the prompt for Gemini
    prompt = f"""Generate a concise and informative social media post for a government announcement.
    The announcement type is: {announcement_type}.

    Here are the details:
    """
    for key, value in details.items():
        prompt += f"- {key.replace('_', ' ').title()}: {value}\n"

    prompt += "\nInclude relevant hashtags. The tone should be appropriate for a government announcement (e.g., urgent for emergencies, celebratory for holidays)."

    try:
        response = gemini_model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print(f"Error generating social media post: {e}")
        return ""

# Example usage of the function
print("\n--- Weather Emergency Post ---")
weather_details = {
    "event": "Severe Thunderstorm Warning",
    "area": "Downtown Cityville",
    "time_period": "Until 8 PM EST",
    "instructions": "Seek shelter immediately. Avoid unnecessary travel."
}
print(generate_social_media_post("weather emergency", weather_details))

print("\n--- Holiday Announcement Post ---")
holiday_details = {
    "holiday_name": "National Day of Service",
    "date": "October 26th",
    "message": "Join us in volunteering and making a difference in our community!"
}
print(generate_social_media_post("holiday", holiday_details))

print("\n--- School Closing Post ---")
school_closing_details = {
    "school_name": "All City Schools",
    "reason": "Inclement weather and icy road conditions",
    "date": "Tomorrow, January 15th",
    "instructions": "Stay safe and warm! Online learning resources available."
}
print(generate_social_media_post("school closing", school_closing_details))


--- Weather Emergency Post ---
<MagicMock name='mock.generate_content().text.strip()' id='139006370035088'>

--- Holiday Announcement Post ---
<MagicMock name='mock.generate_content().text.strip()' id='139006370035088'>

--- School Closing Post ---
<MagicMock name='mock.generate_content().text.strip()' id='139006370035088'>


In [5]:
# EVALUATION
evaluation_data = [
    {"input": "How do I apply for a job?", "expected": "Employment"},
    {"input": "Where can I pay my property taxes?", "expected": "Tax Related"},
    {"input": "Is the library open today?", "expected": "General Information"},
    {"input": "I need an ambulance right now!", "expected": "Emergency Services"},
]


In [6]:
from google.cloud import aiplatform_v1beta1 as eval_api

client = eval_api.EvaluationServiceClient()


In [7]:
def evaluate_classifier(model_function, dataset):
    results = []

    for item in dataset:
        predicted = model_function(item["input"])
        expected = item["expected"]

        results.append({
            "question": item["input"],
            "expected": expected,
            "predicted": predicted,
            "correct": predicted == expected
        })

    return results


In [8]:
# RESULTS

classifier_results = evaluate_classifier(classify_question_gemini, evaluation_data)

for r in classifier_results:
    print(f"Q: {r['question']}")
    print(f"Expected: {r['expected']}")
    print(f"Predicted: {r['predicted']}")
    print(f"Correct: {r['correct']}")
    print("---")


Q: How do I apply for a job?
Expected: Employment
Predicted: Unknown
Correct: False
---
Q: Where can I pay my property taxes?
Expected: Tax Related
Predicted: Unknown
Correct: False
---
Q: Is the library open today?
Expected: General Information
Predicted: Unknown
Correct: False
---
Q: I need an ambulance right now!
Expected: Emergency Services
Predicted: Unknown
Correct: False
---


In [15]:
#ACCURACY

accuracy = sum(r["correct"] for r in classifier_results) / len(classifier_results)
print(f"\nClassifier Accuracy: {accuracy:.2f}")



Classifier Accuracy: 0.00


In [10]:
# RETURNED RESULTS

def classify_question_gemini_v2(question):
    categories = ["Employment", "General Information", "Emergency Services", "Tax Related"]
    prompt = f"""
    You are a government helpdesk assistant.
    Classify the user's question into one of these categories:
    {categories}

    Respond ONLY with the category name.
    Question: {question}
    """

    try:
        response = gemini_model.generate_content(prompt)
        result = response.text.strip()

        return result if result in categories else "Unknown"
    except:
        return "Unknown"


In [11]:
results_v1 = evaluate_classifier(classify_question_gemini, evaluation_data)
results_v2 = evaluate_classifier(classify_question_gemini_v2, evaluation_data)


In [12]:
# VERSION ACCURACY

def compute_accuracy(results):
    return sum(r["correct"] for r in results) / len(results)

print("Prompt Version 1 Accuracy:", compute_accuracy(results_v1))
print("Prompt Version 2 Accuracy:", compute_accuracy(results_v2))


Prompt Version 1 Accuracy: 0.0
Prompt Version 2 Accuracy: 0.0


In [13]:
# Let's Test it Ourselves

def test_input():
    print("=== Manual Classifier Test ===")
    q = input("Enter a question to classify: ")
    print("Category:", classify_question_gemini(q))

    print("\n=== Manual Announcement Test ===")
    t = input("Announcement type (weather emergency, holiday, school closing, etc.): ")

    d = {}
    print("Enter announcement details as key=value pairs. Type 'done' when finished.")
    while True:
        entry = input()
        if entry.lower() == "done":
            break
        if "=" in entry:
            key, value = entry.split("=", 1)
            d[key.strip()] = value.strip()

    print("\nGenerated Post:")
    print(generate_social_media_post(t, d))


In [14]:
test_input()


=== Manual Classifier Test ===
Enter a question to classify: school closings
Category: Unknown

=== Manual Announcement Test ===
Announcement type (weather emergency, holiday, school closing, etc.): done
Enter announcement details as key=value pairs. Type 'done' when finished.
done

Generated Post:
<MagicMock name='mock.generate_content().text.strip()' id='139006370035088'>
